In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from PIL import Image

In [2]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((32, 32)),
        transforms.Normalize((0.5,),(1.0,))     # 과적합을 방지하기 위해서 정규화를 시켜줌. ex. 빨간 것은 사과로 잘못 판단함.

    ]
)

train_data = datasets.MNIST(root= './', train= True, download= True, transform= data_transform)
test_data = datasets.MNIST(root= './', train= False, download= True, transform= data_transform)

In [3]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size= 32, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 32)

In [4]:
data, label = next(iter(train_loader))
print(data.shape)

torch.Size([32, 1, 32, 32])


In [5]:
class Lenet(nn.Module):     # 클래스 이름의 첫 글자는 무조건 대문자를 쓴다.
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 6, kernel_size= 5, stride= 1)
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 16, kernel_size= 5, stride= 1)
        self.conv3 = nn.Conv2d(in_channels= 16, out_channels= 120, kernel_size= 5, stride= 1)
        self.fc1 = nn.Linear(in_features= 120, out_features= 84)
        self.fc2 = nn.Linear(in_features= 84, out_features= 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)       # 6, 28, 28
        x = F.max_pool2d(x, 2, 2)       # 6, 14, 14 Subsampling
        
        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)

        x = self.fc1(x)
        x = F.tanh(x)

        x = self.fc2(x)
       

        return x

model = Lenet()
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
from torchsummary import summary

summary(model, input_size= (1, 32, 32))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0

In [ ]:
lr = 1e-3
optim = Adam(model.parameters(), lr= lr)
epochs = 5
criterion = nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

writer = SummaryWriter()

step = 0
for epoch in range(epochs):
    for data, label in train_loader:
        optim.zero_grad()
        pred = model(data.to(device))
        loss = criterion(pred, label.to(device))
        writer.add_scalar('Loss/train', loss, step)
        step += 1

        loss.backward()
        optim.step()
        
    print(f'{epoch+1} loss: {loss.item():.6f}')
    

1 loss: 0.855470
2 loss: 0.858443
3 loss: 0.807681
4 loss: 0.807971
5 loss: 0.874304
6 loss: 0.797727
7 loss: 0.812084
8 loss: 0.797004
9 loss: 0.815461
10 loss: 0.797027
11 loss: 0.798380
12 loss: 0.808834
13 loss: 0.800814
14 loss: 0.796650
15 loss: 0.796874
16 loss: 0.796626
17 loss: 0.861566
18 loss: 0.796684
19 loss: 0.796652
20 loss: 0.796667


In [9]:
model.eval()        # 학습용으로 쓰기 위해 가중치를 고정시키는 작업

with torch.no_grad():  
    total_corr = 0
    for images, labels in test_loader:
        # print(images.shape)
        images = images.to(device)
        labels = labels.to(device)

        preds = model(images)

        _, pred = torch.max(preds.data, 1)
        total_corr = total_corr + (pred == labels).sum().item()
                
    print(f'정확도: {total_corr/len(test_data.targets)}')

정확도: 0.9883


1. 모델 평가코드작성
2. png이미지를 불러온다.
3. 이미지를 학습할때와 같이 전처리 해준다.
4. 모델에 넣고 추론한다.
5. [ -1, -1, -1, -1, 1.34 , , , ]

In [15]:
model.eval()        # 평가 모드로 전환
with torch.no_grad():
    total_corr = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        preds = model(images)
        _, pred = torch.max(preds.data, 1)
        total_corr += (pred == labels).sum().item()
    print(f'정확도: {total_corr/len(test_data.targets):.4f}')

정확도: 0.9883


In [13]:
input_image = Image.open('../data/4.jpg').convert('L')
input_image = data_transform(input_image)
input_image.shape

torch.Size([1, 32, 32])

In [14]:
with torch.no_grad():
    # print(images.shape)
    input_image = input_image.to(device)


    preds = model(input_image)

    _, pred = torch.max(preds.data, 1)
    print(pred.item())


4


In [17]:
from PIL import Image

# 1. 이미지 불러오기 (경로는 실제 파일 위치로 수정)
input_image = Image.open('../data/4.jpg').convert('L')

# 2. 전처리 (학습 때와 동일)
input_image = data_transform(input_image)

# 3. 배치 차원 추가
input_image = input_image.unsqueeze(0)  # shape: [1, 1, 32, 32]

# 4. 디바이스 이동
input_image = input_image.to(device)

# 5. 추론
model.eval()
with torch.no_grad():
    preds = model(input_image)
    print(preds)  # [ -1, -1, -1, -1, 1.34, ... ]
    _, pred = torch.max(preds.data, 1)
    print(f'예측 결과: {pred.item()}')

tensor([[-1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000]], device='cuda:0')
예측 결과: 4
